##Connect to database! 
###Get Neighborhood Data

In [2]:
#send_Example.py 
import pandas as pd
import json
import psycopg2
import random

In [6]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
        SELECT
                DISTINCT
                ST_asGEOJSON(wkb_geometry) geometry
                , sfn.geoid10

                FROM sf_neighb sfn
                ;"""

df = pd.read_sql(map_query, conn)


In [10]:
df.to_csv('sql_shapes.csv')

note to self: start here with the rest of the code. 
Exported the code so that I could load it up on the mac mini and read it WAY FASTER

In [3]:
df = pd.read_csv('sql_shapes.csv')

In [12]:
df.head()

,Unnamed: 0,geometry,geoid10,hexcolors
0,0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002273,#99CC33
1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002269,#99CC33
2,2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002270,#CCFF00
3,3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002266,#FF0000
4,4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002261,#FF00CC


###Get a Random Color For Each Neighborhood
####Later will encode information by color and hue

In [6]:
def getrandom_color(df, color_file):
    '''takes in a dataframe of gemoetry and geoid, and a list of hex colors
    Creates a new colomn of shape colors. Returns the new dataframe.'''
    #import the colors csv
    cdf = pd.read_csv('colors.csv')
    #make it a list
    colors = cdf[' color'].tolist() 
    num_col = df.shape[0] 
    df['hexcolors'] = [random.choice(colors) for i in xrange(num_col)]
    return df

In [7]:
df_col = getrandom_color(df, 'colors.csv')


In [14]:
df_col.head()


,Unnamed: 0,geometry,geoid10,hexcolors
0,0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002273,#99CC33
1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002269,#99CC33
2,2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002270,#CCFF00
3,3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002266,#FF0000
4,4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002261,#FF00CC


In [22]:
df_col.head()

,geometry,geoid10,hexcolors
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002273,#99CC33
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002269,#99CC33
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002270,#CCFF00
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002266,#FF0000
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-114....",60710251002261,#FF00CC


###Convert DataFrame to JsonFile for Mapping

In [23]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolors']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

list_to_export = [add_properties_geo(row) for idx, row in df_col.iterrows()]

with open('mini_test.json', 'w') as outfile:
    json.dump(list_to_export, outfile)

###stupid experiment that doesn't make sense

In [ ]:
in_file = open('mini_test.json','r')
sf_cong_113 = json.load(in_file)

In [ ]:
def reverse_geojson_coordinates(json_file, output_name):
    '''takes a geo_jason file. Opens it, and reverses the latitude and longitude
    positions so that it is more compatible with mapbox.
    Returns the fixed jason list.'''
    in_file = open(json_file,'r')
    to_fix = json.load(in_file)
    #iterate over every coordinate so I can get the lat and long
    for geoobject in to_fix:
        coords = geoobject['geometry']['coordinates'][0][0]
        newcords = []
        #flip the position of the lat and long 
        for co in coords:
            co = [co[1], co[0]]
            newcords.append(co)
        #insert the new coordinates back into the dictionary
        geoobject['geometry']['coordinates'][0][0] = newcords
    #dump it as a fixed jason object
    with open(output_name, 'w') as outfile:
        json.dump(to_fix, outfile)


In [ ]:
json_file = reverse_geojson_coordinates("mini_test.json", 'messed_up.json')